In [ ]:
import atomap.api as am
import numpy as np
import hyperspy.api as hs
import atomap.dummy_data as dummy_data

s = hs.load('/Users/usuario/Python PM Office/UCL/20231104/L_c23046a_01/DF Stack_11 aligned sum ABSF Filtered RL STEM Inverse.dm3') #introduce first inverse ABF or HAADF
s.plot()

In [ ]:
%matplotlib nbagg
A_positions = am.get_atom_positions(s, separation=15)
sublattice_A = am.Sublattice(A_positions, image=s.data, color='r')
atom_positions_new = am.add_atoms_with_gui(s, A_positions)

In [ ]:
atom_positions_select = am.select_atoms_with_gui(s, atom_positions_new)

In [ ]:
atom_positions_new = am.add_atoms_with_gui(s, atom_positions_select)

In [ ]:
sublattice_A = am.Sublattice(atom_positions_new, s)
sublattice_A.find_nearest_neighbors()
sublattice_A.refine_atom_positions_using_center_of_mass()
sublattice_A.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.4)
sublattice_A.construct_zone_axes(atom_plane_tolerance=0.5)
sublattice_A.plot()
sublattice_A.plot_planes()

In [ ]:
zone_axis_001 = sublattice_A.zones_axis_average_distances[2] #1 introduces vertical zone axes
B_positions = sublattice_A.find_missing_atoms_from_zone_vector(zone_axis_001)

In [ ]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_without_A = remove_atoms_from_image_using_2d_gaussian(sublattice_A.image, sublattice_A)

In [ ]:
sublattice_B = am.Sublattice(B_positions, image_without_A, color='blue')
sublattice_B.construct_zone_axes()
sublattice_B.refine_atom_positions_using_center_of_mass()
sublattice_B.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)
sublattice_B.plot_planes()
atom_lattice = am.Atom_Lattice(image=s.data, name='test', sublattice_list=[sublattice_A, sublattice_B])
atom_lattice.plot()

In [ ]:
s_ABF = hs.load('/Users/usuario/Python PM Office/UCL/20231104/L_c23046a_01/DF Stack_11 aligned sum ABSF Filtered RL STEM.dm3')
s_ABF.plot()

In [ ]:
initial_positions = sublattice_A.atom_positions
sublattice_A2 = am.Sublattice(initial_positions, image=np.divide(1, s_ABF.data), color='r') #if the image is already inversed use  s_ABF.data without image=np.devide 
atom_positions_new = am.add_atoms_with_gui(np.divide(1, s_ABF.data), initial_positions)

In [ ]:
sublattice_A2 = am.Sublattice(atom_positions_new, image=np.divide(1, s_ABF.data), color='r')
sublattice_A2.find_nearest_neighbors()
sublattice_A2.refine_atom_positions_using_center_of_mass()
sublattice_A2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.4)
sublattice_A2.construct_zone_axes()
image_without_A2 = remove_atoms_from_image_using_2d_gaussian(sublattice_A2.image, sublattice_A2)

In [ ]:
initial_positions = sublattice_B.atom_positions
sublattice_B2 = am.Sublattice(initial_positions, image=image_without_A2, color='b')
atom_positions_new = am.add_atoms_with_gui(image_without_A2, initial_positions)

In [ ]:
sublattice_B2 = am.Sublattice(atom_positions_new, image=image_without_A2, pixel_size=4, color='b')
sublattice_B2.find_nearest_neighbors()
sublattice_B2.refine_atom_positions_using_center_of_mass() 
sublattice_B2.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3)#It is better not using them when the image is blur or use smaller percent
sublattice_B2.construct_zone_axes()
sublattice_B2.plot_planes()
sublattice_B2.plot()

In [ ]:
zone_axis_002a = sublattice_B2.zones_axis_average_distances[0]# 0 since the oxygen is parallel to B2
zone_axis_002b = sublattice_B2.zones_axis_average_distances[1]
O1_positions = sublattice_B2.find_missing_atoms_from_zone_vector(zone_axis_002a,vector_fraction=0.5,extend_outer_edges=True, outer_edge_limit=5)
O2_positions = sublattice_B2.find_missing_atoms_from_zone_vector(zone_axis_002b,vector_fraction=0.5,extend_outer_edges=True, outer_edge_limit=5)
O_positions = O1_positions + O2_positions
image_without_AB = remove_atoms_from_image_using_2d_gaussian(sublattice_B2.image, sublattice_B2)

In [ ]:
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g') #or use image_without_AB instead of s. but preferably use s
atom_positions_new = am.add_atoms_with_gui(s_ABF, O_positions) #instead of s I can use image without_AB however sometimes the quality of this image is bad
print(len(O1_positions))
print(len(O2_positions))
print(len(O_positions))

In [ ]:
sublattice_O = am.Sublattice(O_positions, image_without_AB, color='g') #or use s instead of image_without_A. but preferably use image without A
sublattice_O.construct_zone_axes()
sublattice_O.refine_atom_positions_using_center_of_mass()# for oxygen is better to use both fitting methods
sublattice_O.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.3) # the % of the gaussia radious. 

In [ ]:
atom_lattice = am.Atom_Lattice(image=s_ABF.data, name='ABO3', sublattice_list=[sublattice_A, sublattice_B, sublattice_O])
atom_lattice.plot()
atom_lattice.plot(image=s.data)

In [ ]:
#polarization map
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[0]
za1 = sublattice_A.zones_axis_average_distances[1]
za2 = sublattice_A.zones_axis_average_distances[2]
sublattice_A.plot_planes(image=atom_lattice.image)# use this function to reveal the planes and select the appropriate ones. The number of the plane is indicated on the top
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za2, sublattice_O)
s_p_2 = sublattice_A.get_polarization_from_second_sublattice(za1, za2, sublattice_O)
vector_list_1 = s_p.metadata.vector_list
vector_list_2 = s_p_2.metadata.vector_list
vector_list = vector_list_1 + vector_list_2
x, y = [i[0] for i in vector_list], [i[1] for i in vector_list]
u, v = [i[2] for i in vector_list], [i[3] for i in vector_list]

sampling, resolution = 0.008902054, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=resolution,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=150)

In [ ]:
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_O = atom_lattice.sublattice_list[2]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[0] # change this depending on the types of oxygen. (use either [1] and [2] or [0] and [2]
za1 = sublattice_A.zones_axis_average_distances[1]
za2 = sublattice_A.zones_axis_average_distances[2]
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za2, sublattice_O)
s_p_2 = sublattice_A.get_polarization_from_second_sublattice(za1, za2, sublattice_O)
vector_list_1 = s_p.metadata.vector_list
vector_list_2 = s_p_2.metadata.vector_list
vector_list = vector_list_1 + vector_list_2
w_1=vector_list_1
w_2=vector_list_2
w=vector_list

#The number of O atoms that have their polarization calculated is less than the calculated O positions
zone_vector_0 = sublattice_A.zones_axis_average_distances[0] #If columns start first use 1
zone_vector_1 = sublattice_A.zones_axis_average_distances[1]
atom_planes_0 = sublattice_A.atom_planes_by_zone_vector[zone_vector_0]
atom_planes_1 = sublattice_A.atom_planes_by_zone_vector[zone_vector_1]
Num_of_planes_0 = len(atom_planes_0)
Num_of_planes_1 = len(atom_planes_1)
print(Num_of_planes_0)
print(Num_of_planes_1)

rows_1=Num_of_planes_1-2     #number of Oxygen rows -1       Axial Oxygens  Reverse Num_of_planes_0 and Num_of_planes_1 if colums are first in the zone axis list
columns_1=Num_of_planes_0-1  #number of Oxygen columns -2    Axial Oxygens
rows_2=Num_of_planes_1-1     #number of Oxygen rows -2       Equitorial Oxygens
columns_2=Num_of_planes_0-2  #number of Oxygen columns -1    Equitorial Oxygens

# function to return the second element of the w_1 list
def colsort_1():
    def sortSecond(val):
        return val[1] 
  
        #sort each column individualy. 
        # using the second key 
        
    d=rows_1
    p=0
    w_1.sort()#sort the list to the first element of each item which is the row.
    for i in range(columns_1): #number of columns
        s=w_1[p:d] #Select all the elemnts of the first column
        s.sort(key = sortSecond) #Sort the element from the top to bottom of the column
        w_1[p:d]=s # replace the elements of the list with the sorted ones
        p=p+rows_1
        d=d+rows_1
colsort_1()


# function to return the second element of the w_2 list
def colsort_2():
    def sortSecond(val):
        return val[1] 
  
        #sort each column individualy. 
        # using the second key 
   
    d=rows_2
    p=0
    w_2.sort()#sort the list to the first element of each item which is the row.
    for i in range(columns_2): #number of columns
        s=w_2[p:d] #Select all the elemnts of the first column
        s.sort(key = sortSecond) #Sort the element from the top to bottom of the column
        w_2[p:d]=s # replace the elements of the list with the sorted ones
        p=p+rows_2
        d=d+rows_2
colsort_2() 
w = w_1 + w_2
w

In [ ]:
#**********Dipole filter**********

colsort_1()
x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w_1], [i[3] for i in w_1]

#print(len(u)) #check the length of the vector so as to create a list with the same size

rows= rows_1 # number of Oxygen rows
columns= columns_1
elements= rows*columns # number of Oxygen atoms
resolution= 0.008902054

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]
#bottom multiplication
w_1.append(w_1.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u_1 = [0.3*u[t]+0.1*i[2] for i,t in zip(w_1,list_n)]
v_1 = [0.3*v[t]+0.1*i[3] for i,t in zip(w_1,list_n)]

#up multiplication
colsort_1()
for b in range(elements-1):
    w_1.append(w_1.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u_1 = [u_1[t]+0.1*i[2] for i,t in zip(w_1,list_n)]
v_1 = [v_1[t]+0.1*i[3] for i,t in zip(w_1,list_n)]

#side +1 column
colsort_1()
for b in range(rows_1):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_1.append(w_1.pop(0))
u_1 = [u_1[t]+0.15*i[2] for i,t in zip(w_1,list_n)]
v_1 = [v_1[t]+0.15*i[3] for i,t in zip(w_1,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_1.append(w_1.pop(0))
u_1 = [u_1[t]+0.1*i[2] for i,t in zip(w_1,list_n)]
v_1 = [v_1[t]+0.1*i[3] for i,t in zip(w_1,list_n)]

#side -1 column
colsort_1()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_1.append(w_1.pop(0))
u_1 = [u_1[t]+0.15*i[2] for i,t in zip(w_1,list_n)]
v_1 = [v_1[t]+0.15*i[3] for i,t in zip(w_1,list_n)]

#side -2 column
colsort_1()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_1.append(w_1.pop(0))
u_1 = [u_1[t]+0.1*i[2] for i,t in zip(w_1,list_n)]
v_1 = [v_1[t]+0.1*i[3] for i,t in zip(w_1,list_n)]
colsort_1()

#print(w)  #use this function to check if the columns are sepparated correctly
#for diagonal multiplication a combination between bottom and side multiplication is needed. Specifically "for b in range(n+1)"



#**********Dipole filter**********



colsort_2()
x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w_2], [i[3] for i in w_2]

#print(len(u)) #check the length of the vector so as to create a list with the same size

rows= rows_2 # number of Oxygen rows
columns= columns_2
elements=rows*columns # number of Oxygen atoms

#the list below is used to interate all over the vectors in the u and v lists.
list_n_2 = [*range(elements)]

#bottom multiplication
w_2.append(w_2.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u_2 = [0.3*u[t]+0.1*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [0.3*v[t]+0.1*i[3] for i,t in zip(w_2,list_n_2)]

#up multiplication
colsort_2()
for b in range(elements-1):
    w_2.append(w_2.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u_2 = [u_2[t]+0.1*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [v_2[t]+0.1*i[3] for i,t in zip(w_2,list_n_2)]

#side +1 column
colsort_2()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_2.append(w_2.pop(0))
u_2 = [u_2[t]+0.15*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [v_2[t]+0.15*i[3] for i,t in zip(w_2,list_n_2)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_2.append(w_2.pop(0))
u_2 = [u_2[t]+0.1*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [v_2[t]+0.1*i[3] for i,t in zip(w_2,list_n_2)]

#side -1 column
colsort_2()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_2.append(w_2.pop(0))
u_2 = [u_2[t]+0.15*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [v_2[t]+0.15*i[3] for i,t in zip(w_2,list_n_2)]

#side -2 column
colsort_2()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w_2.append(w_2.pop(0))
u_2 = [u_2[t]+0.1*i[2] for i,t in zip(w_2,list_n_2)]
v_2 = [v_2[t]+0.1*i[3] for i,t in zip(w_2,list_n_2)]

u_3=u_1+u_2
v_3=v_1+v_2
sampling, units =  resolution, 'nm' #image calibration

tml.plot_polarisation_vectors(x, y, u_3, v_3, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='r', overlay=True, monitor_dpi=250)
tml.plot_polarisation_vectors(x, y, u_3, v_3, image=atom_lattice.image,
                          plot_style="polar_colorwheel",
                          unit_vector=False, overlay=False,
                          save=None, monitor_dpi=250)

tml.plot_polarisation_vectors(x, y, u_3, v_3, image=atom_lattice.image, remove_vectors=True,
                          unit_vector=True, plot_style='contour',
                          overlay=True, pivot='middle', save=None,
                          cmap='cet_colorwheel', alpha=0.5,
                          monitor_dpi=250, vector_rep="angle",
                          antialiased=True, degrees=True)


In [ ]:
# filtered averaging
import statistics
import numpy
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss
def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

resolution = 0.008902054
g=0;
c=0;
stdvp=list=[]  #list for stdv of horizontal displacement
stdvq=list=[]  #list for stdv of vertical displacement



#Displacement per row of equitarial oxygens
print("                        Equitorial Oxygen                           ")
print("        Δx                 Δy               STDVx              STDVy")
for q in range(rows_1): #this range is the number of rows
    t=q
    for i in range(columns_1): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+rows_1     #this range is the number of rows
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    
       # stdvq.append(z)
   ##  STDVp=numpy.std(arrp, axis=0)
   # arrq = numpy.array([q])
  #  e=numpy.std(arrq, axis=0)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)

    l=(1000*c*resolution)/columns_1  #horizontal x average value in pm
    a=(1000*g*resolution)/columns_1  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0    

#Displacement per row of Apical oxygens
print("                          Apical Oxygen                             ")
print("        Δx                 Δy               STDVx              STDVy")
for q in range(rows_2): #this range is the number of rows
    t=q
    for i in range(columns_2): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+rows_2     #this range is the number of rows
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    
       # stdvq.append(z)
   ##  STDVp=numpy.std(arrp, axis=0)
   # arrq = numpy.array([q])
  #  e=numpy.std(arrq, axis=0)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)

    l=(1000*c*resolution)/columns_2  #horizontal x average value in pm
    a=(1000*g*resolution)/columns_2  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0   







colsort_1()
colsort_2()
w=w_1+w_2
vector_list=w
x, y = [i[0] for i in vector_list], [i[1] for i in vector_list]
u, v = [i[2] for i in vector_list], [i[3] for i in vector_list]

#Displacement per column of Equitorial Oxygen
print("             Equitorial Oxygen displacement per column              ")
print("        Δx                 Δy               STDVx              STDVy")
for q in range(columns_1): #this range is the number of rows
    for i in range(rows_1): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+1    #got to next row
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)
    l=(1000*c*resolution)/rows_1  #horizontal x average value in pm
    a=(1000*g*resolution)/rows_1  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0    

#Displacement per column of Apical Oxygen
print("               Apical Oxygen displacement per column                ")
print("        Δx                 Δy               STDVx              STDVy")
t=columns_1*rows_1;
for q in range(columns_2): #this range is the number of rows
    for i in range(rows_2): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+1    #got to next row
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)
    l=(1000*c*resolution)/rows_2  #horizontal x average value in pm
    a=(1000*g*resolution)/rows_2  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0    

In [ ]:
#Displacement per column